# Politics in Virgina

In [2]:
SCHOOL="virgina"
SUBJECT="opinion"
START_YEAR=2010
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [3]:
%cd ..

/home/alexandersuen/diversity-colleges


In [4]:
%cd src

/home/alexandersuen/diversity-colleges/src


In [5]:
import sys
sys.path.append('src')
import ouraws
import nltk
import ourgraphs
import textutil

In [6]:
%cd ..

/home/alexandersuen/diversity-colleges


In [12]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [13]:
df.shape

(4893, 6)

In [14]:
df.head(2)

,title,url,body,year,month,day
0,EDITORIAL: Students — vote for The Community C...,https://www.cavalierdaily.com/article/2023/02/...,EDITORIAL: Students — vote for The Community C...,2023,2,28
1,LETTER: Turn out for student democracy,https://www.cavalierdaily.com/article/2023/02/...,LETTER: Turn out for student democracy\nWhen I...,2023,2,27


In [15]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")
#BIPARTISAN_API_KEY = "gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=="
print(BIPARTISAN_API_KEY)
BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g==


In [16]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

1.492847


In [17]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009])

In [18]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2010 ... 425
2010	425	25.393920999999803	0.05975040235294071
Year: 2011 ... 486
2011	485	275.60471100000007	0.5682571360824744
Year: 2012 ... 449
2012	442	11.59844100000027	0.026240816742082056
Year: 2013 ... 528
2013	510	-44.71316199999986	-0.08767286666666639
Year: 2014 ... 574
2014	570	312.3592180000001	0.5479986280701756
Year: 2015 ... 535
2015	526	271.63047700000004	0.5164077509505705
Year: 2016 ... 427
2016	415	-502.1438169999999	-1.209985101204819
Year: 2017 ... 367
2017	359	-524.39807	-1.4607188579387185
Year: 2018 ... 242
2018	237	-186.91843900000006	-0.7886853966244728
Year: 2019 ... 270
2019	265	-183.85450699999998	-0.6937905924528301
Year: 2020 ... 206
2020	200	-466.5333129999999	-2.3326665649999994
Year: 2021 ... 157
2021	151	-372.1685379999999	-2.464692304635761
Year: 2022 ... 207
2022	205	-427.02720499999975	-2.0830595365853646


In [ ]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)

In [ ]:
# ASYNCHRONOUS METHOD : should be a lot faster
import asyncio

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year